In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
import os

os.getcwd()


'c:\\Users\\nik\\Desktop\\Berkeley_Projects\\LoadLanguageModel\\bin'

In [2]:
df = pd.read_csv('../data\MFRED_Aggregates_15min_2019Q1-Q4.csv', index_col=0, parse_dates=True).filter(regex=r'^.*kW$')
df = df.iloc[:, 1:]
scaler = MinMaxScaler()

df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns, index=df.index)

df = df.resample('1H').mean()
df = df.stack().reset_index().rename(columns={'level_0': 'timestamp', 'level_1': 'apartment', 0: 'kW'})

# drop the _Kw from the apartment names

df['apartment'] = df['apartment'].str.replace('_kW', '')



In [3]:
# reset index and call the first column apartment
df_meta = pd.read_csv("../data/meta_data.csv", index_col=0).iloc[:-1, :]
df_meta = df_meta.reset_index().rename(columns={'AP ': 'apartment'})
df_meta.head()

,Apt. group,Time-averaged real power (σ) [W],Number of bedrooms (σ),Number of all rooms (σ),Apt. area (σ) [m2]
0,AG01,42,1.2,3.2,62
1,AG02,81,1.3,3.9,77
2,AG03,105,1.3,3.3,65
3,AG04,119,1.1,3.1,58
4,AG05,141,1.5,4.2,91


In [4]:

scaler_meta = MinMaxScaler(feature_range=(-1, 1))

df_meta[df_meta.columns[1:]] = scaler_meta.fit_transform(df_meta[df_meta.columns[1:]]) # scale the meta data

df_meta.head()

,Apt. group,Time-averaged real power (σ) [W],Number of bedrooms (σ),Number of all rooms (σ),Apt. area (σ) [m2]
0,AG01,-1.000000,-0.888889,-0.941176,-0.925234
1,AG02,-0.935697,-0.777778,-0.529412,-0.644860
2,AG03,-0.896125,-0.777778,-0.882353,-0.869159
3,AG04,-0.873042,-1.000000,-1.000000,-1.000000
4,AG05,-0.836768,-0.555556,-0.352941,-0.383178


In [5]:


df_meta.rename(columns={'Apt. group': 'apartment'}, inplace=True)
df.set_index('DateTimeUTC', inplace=True)


In [6]:
def timeseries_dataframe_pivot(df):
    df['date'] = df.index.date
    df['time'] = df.index.time

    df_pivot = df.pivot(index='date', columns='time')

    n_days, n_timesteps = df_pivot.shape

    df_pivot.dropna(thresh = n_timesteps // 5, inplace=True)

    df_pivot = df_pivot.fillna(method='ffill', axis = 0)

    df_pivot = df_pivot.droplevel(0, axis=1)

    df_pivot.columns.name = None

    return df_pivot


def data_augmentation(df, n_samples=1000, noise=0.1, scale=0.1, shift_range=(-2,2)):
    df = df.copy()
    df_list = []
    for i in range(n_samples):
        df_noise = df + np.random.normal(0, noise, df.shape)
        shift_value = np.random.uniform(shift_range[0], shift_range[1])
        df_shift = pd.DataFrame(np.roll(df_noise.values, int(shift_value), axis=1), columns=df.columns, index=df.index)
        df_scale = df_shift * (1 + np.random.normal(0, scale, df.shape))
        # set negative values to zero
        df_scale[df_scale < 0] = 0
        # set values above 1 to 1
        df_scale[df_scale > 1] = 1
        df_list.append(df_scale)
    return pd.concat(df_list, axis=0)





In [7]:
# Data augmentation and merging with meta data

list_df_ready = []

for name, group in df[["kW", "apartment"]].groupby('apartment'):
    df_pivot = timeseries_dataframe_pivot(group[["kW"]]).iloc[1:, :]
    df_pivot_augmented = data_augmentation(df_pivot, n_samples=20, noise=0.02, scale=0.02)
    #df_pivot_augmented = df_pivot
    df_meta_data_columns = df_meta.loc[df_meta['apartment'] == name].iloc[:,2:]
    df_pivot_augmented[["bedrooms", "rooms", "area"]] = np.repeat(df_meta_data_columns.values, df_pivot_augmented.shape[0], axis=0)
    list_df_ready.append(df_pivot_augmented)

df_ready = pd.concat(list_df_ready, axis=0)



In [9]:
df_ready.to_csv("../data/apartment_load_data_hourly_not_augmented.csv")

In [228]:
365*26

9490

In [12]:
df_ready.iloc[:, :-3].to_csv("../data/apartment_autoencoder.csv")